1. Simulate noisy (uniform, multiplicative) measurements from full model with "true" parameters
2. Choose a reduced model to fit
3. Run Levenberg-Marquardt least squares optimisation to find "best fit" parameters (note the mismatch between noise and cost function types)
4. Compute log Hessian around best fit to 
5. Compute error in parameters

Rinse and repeat

During nbinclude make sure CURDATAPOINT exists in the workspace outside, and it changes

In [1]:
# Helper functions

# Get a timestamp in useful format
timestamp()=Dates.format(now(),"_yyyymmddTHHMMSSss")

include("../BK_functions/bk_setup_script.jl")


8-element Array{Float64,1}:
  2.2e-6
  0.42  
  0.1026
  0.58  
  3.9e-5
  6.16  
 30.4   
  2.0   

In [ ]:
# Load simulated data from fig_A1
using JLD
files = readdir("saves")

all_ydata_orig = Array(Any, 110)
all_ydata_log = Array(Any, 110)
all_filenames = Array(Any, 110)

counter = 0
for i in files
    if contains(i, "1110T") && counter < 100 # for 10% error version
        ydata_orig, ydata_log = load("saves/"*i, "ydata_orig", "ydata_log")
        counter += 1
        all_ydata_orig[counter] = ydata_orig
        all_ydata_log[counter] = ydata_log
        all_filenames[counter] = i
    end
end

In [ ]:
# Load original fits for reduced models from fig_3 and fig_4
outp = load("saves/figure3_smallVnorm.jld", "outp");
outp_log10 = load("saves/figure4.jld", "outp_log10");

# Choose models and corresponding phi fit
model_orig_id = 11; # The one in Fig 3B and Fig 6C (3 reduction steps)
model_log_id = 54; # The one in Fig 4C (2 reduction steps)

phi0_orig = exp(outp[1][2,1])
phi0_log = exp(outp_log10[1][4,1])

# CURDATAPOINT: global variable from outside - which data file to use as input or give in here:
# CURDATAPOINT = 5

## Fit noisy original assay and compute Hessian around best fit

In [ ]:
# Optimise phi to fit the noisy data well
using LsqFit

ydata_orig = all_ydata_orig[CURDATAPOINT] 
cur_time = timestamp();

# Store multiple fits (different initial point for LM, takes around 5 sec / fit)
all_fits = Array(Any, 24)
all_costs = Array(Any, length(all_fits))

for i1 = 1:length(all_fits)
    if i1==1
        phi_init = phi0_orig;
    else
        phi_init = phi0_orig.*(1+(2*rand(size(phi0_orig))-1)*0.9) # Add up to 90% noise to initial params
    end
    all_fits[i1] = curve_fit((x,p) -> BK_simulator(exp(p), x, model_id=model_orig_id), hcat(x_grid...), ydata_orig, log(phi_init));
    all_costs[i1] = sum(all_fits[i1].resid.^2);
end

# Find the best fit
phi1_orig = exp(all_fits[findmin(all_costs)[2]].param);


# Compute the Hessian around best fit
x_loc = hcat(x_grid...);
model_cur(phi::AbstractArray, x::AbstractArray) = BK_simulator(phi, x, model_id=model_orig_id)[:]
data_vec = ydata_orig;
f_cost = phi_var -> sum((model_cur(phi_var, x_loc)[:].-data_vec).^2); # Current cost function (quadratic)
g_cost = phi_var -> log_deriv_wrapper(f_cost, phi_var); # Define which parameters we're taking in log space ([default: all])
f_res = phi_var -> (model_cur(phi_var, x_loc)[:].-data_vec) # Return residuals
g_res = phi_var -> log_deriv_wrapper(f_res, phi_var, log_specific=log_specific);

# Determine the hyperellipsoid around our best fit:
hess_orig = ForwardDiff.hessian(g_cost, log(phi1_orig)); #log hessian
D_orig, V_orig = eig(hess_orig)

In [ ]:
[phi0_orig phi1_orig]

## Fit noisy log assay and compute Hessian around best fit

In [ ]:
# Store multiple fits (different initial point for LM, takes around 5 sec / fit)
all_fits = Array(Any, 24)
all_costs = Array(Any, length(all_fits))

ydata_log = log10(exp(all_ydata_log[CURDATAPOINT])) # note: In fig A1 we used normal log instead of log10 for both data and model to be fitted

for i1 = 1:length(all_fits)
    if i1==1
        phi_init = phi0_log;
    else
        phi_init = phi0_log.*(1+(2*rand(size(phi0_log))-1)*0.9) # Add up to 90% noise to initial params
    end
    all_fits[i1] = curve_fit((x,p) -> BK_simulator(exp(p), x, model_id=model_log_id), hcat(x_grid...), ydata_log, log(phi_init)); # abs needed due to weird julia error, BK_simulator has an output range between 0 and 1
    all_costs[i1] = sum(all_fits[i1].resid.^2);
end

# Find the best fit
phi1_log = exp(all_fits[findmin(all_costs)[2]].param);

# Compute the Hessian around best fit
x_loc = hcat(x_grid...);
model_cur(phi::AbstractArray, x::AbstractArray) = BK_simulator(phi, x, model_id=model_log_id)[:]
data_vec = ydata_log;
f_cost = phi_var -> sum((model_cur(phi_var, x_loc)[:].-data_vec).^2); # Current cost function (quadratic)
g_cost = phi_var -> log_deriv_wrapper(f_cost, phi_var); # Define which parameters we're taking in log space ([default: all])
f_res = phi_var -> (model_cur(phi_var, x_loc)[:].-data_vec) # Return residuals
g_res = phi_var -> log_deriv_wrapper(f_res, phi_var, log_specific=log_specific);

# Determine the hyperellipsoid around our best fit:
hess_log = ForwardDiff.hessian(g_cost, log(phi1_log)); #log hessian of log assay
D_log, V_log = eig(hess_log)

In [ ]:
[phi0_log phi1_log]

#save results - 10% version (original)
save("saves/figure_A2_hess_"*string(CURDATAPOINT)*".jld", "hess_orig", hess_orig, "D_orig", D_orig,
"hess_log", hess_log, "D_log", D_log, 
"phi1_orig", phi1_orig, "phi1_log", phi1_log, "phi0_orig", phi0_orig, "phi0_log", phi0_log,
"ydata_orig", ydata_orig, "ydata_log", ydata_log, "A1_filename", all_filenames[CURDATAPOINT])

In [ ]:
#save results - 10% version (original)
save("saves/figure_A2_hess_11_origmodel_54_logmodel_"*string(CURDATAPOINT)*".jld", "hess_orig", hess_orig, "D_orig", D_orig,
"hess_log", hess_log, "D_log", D_log, 
"phi1_orig", phi1_orig, "phi1_log", phi1_log, "phi0_orig", phi0_orig, "phi0_log", phi0_log,
"ydata_orig", ydata_orig, "ydata_log", ydata_log, "A1_filename", all_filenames[CURDATAPOINT])